In [6]:
!pip install protein-bert
!pip install gdown

In [ ]:
!gdown --folder --id '1MMvRjjeR1Hz6hw08WZhZwLI-89pUpYtc'

In [1]:
import os
import tensorflow as tf
from Bio import SeqIO
import os
import pandas as pd
import os
import pandas as pd
from IPython.display import display
from tensorflow import keras
from sklearn.model_selection import train_test_split
from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, finetune, evaluate_by_len
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs
import numpy as np
from proteinbert.finetuning import encode_categorical_Y
from proteinbert.finetuning import encode_Y
from proteinbert.finetuning import encode_dataset

In [2]:
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [3]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
#import tensorflow as tf

#tf.config.threading.set_intra_op_parallelism_threads(64)
#tf.config.threading.set_inter_op_parallelism_threads(64)

In [4]:
list_df = []
for file_name in os.listdir('TriTrypDB'):
    record_dict = SeqIO.to_dict(SeqIO.parse('TriTrypDB/'+file_name, 'fasta'))
    
    temp_dict = {}
    for n in record_dict:
        temp_dict[n]={
            'seq':str(record_dict[n].seq),
            'desc':record_dict[n].description,
        }
    record_df = pd.DataFrame.from_dict(temp_dict,orient='index')
    list_df.append(record_df)
list_df=pd.concat(list_df)
print(list_df.shape)
list_df.head()

(38465, 2)


,seq,desc
C4B63_1g1002-t42_1-p1,MAMMMTGRVLLVCALCVLWCGTSGRCDDAKQKELLDQRSVSPLGGE...,C4B63_1g1002-t42_1-p1 | transcript=C4B63_1g100...
C4B63_1g1003-t42_1-p1,MMTCRLLCALLVLALCCCPSVSAAGTQEAVKEPSKSTTTNDTTKAQ...,C4B63_1g1003-t42_1-p1 | transcript=C4B63_1g100...
C4B63_1g1004-t42_1-p1,MAMMMTGRVLLVCALCVLWCGAGGVYARYLDNNPVGGCMASGGFGA...,C4B63_1g1004-t42_1-p1 | transcript=C4B63_1g100...
C4B63_1g1025-t42_1-p1,MAMMMTGRVLLVCALCVLWCGAGEGGAEMNDETLVGSQHLTGVGEK...,C4B63_1g1025-t42_1-p1 | transcript=C4B63_1g102...
C4B63_1g1031-t42_1-p1,MVLKRLLWVWTPHPHQYAYRSMRTTTMQLAHLIHEVEHNRNHYFQV...,C4B63_1g1031-t42_1-p1 | transcript=C4B63_1g103...


In [5]:
list_df['len_seq']=list_df.seq.str.len()

In [6]:
#list_df.to_pickle('TriTrypDB.pkl',compression='gzip')

In [7]:
#list_df_500 = list_df[list_df['len_seq']<500]
#list_df_500.shape

In [8]:
list_df_1000 = list_df[list_df['len_seq']<1000]
list_df_1000.shape

(34112, 3)

In [9]:
def get_models(size = 1024):
    df = pd.read_csv('label_to_family.csv',index_col=[0])
    unique_labels = sorted(set(df['label'].unique()))
    label_dict = dict(zip(df['label'],df['FamilyId']))
    OUTPUT_TYPE = OutputType(False, 'categorical')
    OUTPUT_SPEC = OutputSpec(OUTPUT_TYPE, unique_labels)

    pretrained_model_generator, input_encoder = load_pretrained_model(
    local_model_dump_dir='',
    validate_downloading=False)

    # get_model_with_hidden_layers_as_outputs gives the model output access to the hidden layers (on top of the output)
    model_generator = FinetuningModelGenerator(
        pretrained_model_generator, 
        OUTPUT_SPEC, 
        pretraining_model_manipulation_function = get_model_with_hidden_layers_as_outputs,
        dropout_rate = 0.5)


    fold=0
    fine_tuned_model_0 = model_generator.create_model(size)
    fine_tuned_model_0.load_weights('fine_tuned_model_other_fold_{fold}.h5'.format(fold=fold))

    fold=1
    fine_tuned_model_1 = model_generator.create_model(size)
    fine_tuned_model_1.load_weights('fine_tuned_model_other_fold_{fold}.h5'.format(fold=fold))

    fold=2
    fine_tuned_model_2 = model_generator.create_model(size)
    fine_tuned_model_2.load_weights('fine_tuned_model_other_fold_{fold}.h5'.format(fold=fold))

    fold=3
    fine_tuned_model_3 = model_generator.create_model(size)
    fine_tuned_model_3.load_weights('fine_tuned_model_other_fold_{fold}.h5'.format(fold=fold))

    fold=4
    fine_tuned_model_4 = model_generator.create_model(size)
    fine_tuned_model_4.load_weights('fine_tuned_model_other_fold_{fold}.h5'.format(fold=fold))    

    models = [
        fine_tuned_model_0,
        fine_tuned_model_1,
        fine_tuned_model_2,
        fine_tuned_model_3,
        fine_tuned_model_4,
    ]
    return models, input_encoder, OUTPUT_SPEC, label_dict

models, input_encoder, OUTPUT_SPEC, label_dict = get_models()

2022-06-09 09:58:44.029505: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-09 09:58:44.029643: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c6100-19-2.compute.dundee.ac.uk): /proc/driver/nvidia/version does not exist
2022-06-09 09:58:44.173841: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/homes/mtinti/miniconda3/envs/proteinBert2/lib/python3.10/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [10]:
#[list_df_1000['seq'].str.len()<500]

In [11]:
#temp=temp.reset_index(drop=True)
X, y_true, sample_weights = encode_dataset(list_df_1000['seq'], [50 for n in range(list_df_1000.shape[0])], input_encoder, OUTPUT_SPEC, \
                seq_len = 512*2, needs_filtering = False)

In [12]:
from tqdm.auto import tqdm
for index, model in tqdm(enumerate(models)):
    preds = model.predict(X, batch_size = 32)
    index_sorted = np.argsort(preds, axis=1)
    
    labels_1 = pd.Series(index_sorted[:,-1])
    labels_2 = pd.Series(index_sorted[:,-2])
    
    family_1 = labels_1.map(label_dict)
    family_2 = labels_2.map(label_dict)
    
    prob_1 = preds[np.arange(len(X[0])), labels_1]
    prob_2 = preds[np.arange(len(X[0])), labels_2]
    
    list_df_1000['model_'+str(index)+'_labels_1']=labels_1.values
    list_df_1000['model_'+str(index)+'_family_1']=family_1.values
    list_df_1000['model_'+str(index)+'_prob_1']=prob_1
    
    #list_df_500['model_'+str(index)+'_labels_2']=labels_2
    #list_df_500['model_'+str(index)+'_family_2']=family_2
    #list_df_500['model_'+str(index)+'_prob_2']=prob_2
    
    #temp['preds_model_'+str(index)]

/homes/mtinti/miniconda3/envs/proteinBert2/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
0it [00:00, ?it/s]/tmp/448802.1.all.q/ipykernel_10513/3560440382.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  list_df_1000['model_'+str(index)+'_labels_1']=labels_1.values
/tmp/448802.1.all.q/ipykernel_10513/3560440382.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

In [23]:
def select(X):
    temp = []
    items = list(set(X))
    to_count = list(X)
    #print(temp,items,to_count)
    for n in items:
        temp.append((to_count.count(n),n))
    temp.sort()
    #print(temp)
    return temp[-1][-1]


def vote_mean(X):
    #print(X)
    majority = X['vote']
    score = []
    for a,b in zip(X[['model_0_family_1','model_1_family_1','model_2_family_1','model_3_family_1','model_4_family_1']],
                   X[['model_0_prob_1','model_1_prob_1','model_2_prob_1','model_3_prob_1','model_4_prob_1']]):
        if a==majority:
            score.append(b)
    return np.mean(score)
                   


def get_dict_from_desc(X):
    temp_dict = {}
    item_list= X.split('|')
    #print(item_list)
    for item in item_list:
        if '=' in item:
            key = item.split('=')[0].strip()
            value = item.split('=')[1].strip()
            temp_dict[key]=value
            #print(key,value)
    return temp_dict

def get_desc(X, target):
    temp_dict = get_dict_from_desc(X)
    #print(temp_dict)
    return temp_dict[target]


In [24]:
list_df_1000['vote']=list_df_1000[['model_0_family_1','model_1_family_1','model_2_family_1','model_3_family_1','model_4_family_1']].apply(select,axis=1)
list_df_1000['vote_mean']=list_df_1000.apply(vote_mean,axis=1)
list_df_1000.head()

/tmp/448802.1.all.q/ipykernel_10513/1914304013.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  list_df_1000['vote']=list_df_1000[['model_0_family_1','model_1_family_1','model_2_family_1','model_3_family_1','model_4_family_1']].apply(select,axis=1)
/tmp/448802.1.all.q/ipykernel_10513/1914304013.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  list_df_1000['vote_mean']=list_df_1000.apply(vote_mean,axis=1)


,seq,desc,len_seq,model_0_labels_1,model_0_family_1,model_0_prob_1,model_1_labels_1,model_1_family_1,model_1_prob_1,model_2_labels_1,model_2_family_1,model_2_prob_1,model_3_labels_1,model_3_family_1,model_3_prob_1,model_4_labels_1,model_4_family_1,model_4_prob_1,vote,vote_mean
C4B63_1g1002-t42_1-p1,MAMMMTGRVLLVCALCVLWCGTSGRCDDAKQKELLDQRSVSPLGGE...,C4B63_1g1002-t42_1-p1 | transcript=C4B63_1g100...,380,110,Other,1.0,110,Other,1.000000,110,Other,1.00000,110,Other,1.000000,110,Other,1.000000,Other,1.00000
C4B63_1g1003-t42_1-p1,MMTCRLLCALLVLALCCCPSVSAAGTQEAVKEPSKSTTTNDTTKAQ...,C4B63_1g1003-t42_1-p1 | transcript=C4B63_1g100...,224,110,Other,1.0,110,Other,1.000000,110,Other,1.00000,110,Other,1.000000,110,Other,1.000000,Other,1.00000
C4B63_1g1004-t42_1-p1,MAMMMTGRVLLVCALCVLWCGAGGVYARYLDNNPVGGCMASGGFGA...,C4B63_1g1004-t42_1-p1 | transcript=C4B63_1g100...,420,110,Other,1.0,110,Other,1.000000,110,Other,1.00000,110,Other,1.000000,110,Other,1.000000,Other,1.00000
C4B63_1g1025-t42_1-p1,MAMMMTGRVLLVCALCVLWCGAGEGGAEMNDETLVGSQHLTGVGEK...,C4B63_1g1025-t42_1-p1 | transcript=C4B63_1g102...,479,110,Other,1.0,110,Other,1.000000,110,Other,1.00000,110,Other,1.000000,110,Other,1.000000,Other,1.00000
C4B63_1g1031-t42_1-p1,MVLKRLLWVWTPHPHQYAYRSMRTTTMQLAHLIHEVEHNRNHYFQV...,C4B63_1g1031-t42_1-p1 | transcript=C4B63_1g103...,761,110,Other,1.0,110,Other,0.999886,110,Other,0.99997,110,Other,0.999874,110,Other,0.999871,Other,0.99992


In [25]:
list_df_1000['organism']=list_df_1000['desc'].apply(get_desc,args=('organism',))
list_df_1000['gene_product']=list_df_1000['desc'].apply(get_desc,args=('gene_product',))
list_df_1000['transcript_product']=list_df_1000['desc'].apply(get_desc,args=('transcript_product',))
list_df_1000.head()

/tmp/448802.1.all.q/ipykernel_10513/3364339408.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  list_df_1000['organism']=list_df_1000['desc'].apply(get_desc,args=('organism',))
/tmp/448802.1.all.q/ipykernel_10513/3364339408.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  list_df_1000['gene_product']=list_df_1000['desc'].apply(get_desc,args=('gene_product',))
/tmp/448802.1.all.q/ipykernel_10513/3364339408.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

,seq,desc,len_seq,model_0_labels_1,model_0_family_1,model_0_prob_1,model_1_labels_1,model_1_family_1,model_1_prob_1,model_2_labels_1,...,model_3_family_1,model_3_prob_1,model_4_labels_1,model_4_family_1,model_4_prob_1,vote,vote_mean,organism,gene_product,transcript_product
C4B63_1g1002-t42_1-p1,MAMMMTGRVLLVCALCVLWCGTSGRCDDAKQKELLDQRSVSPLGGE...,C4B63_1g1002-t42_1-p1 | transcript=C4B63_1g100...,380,110,Other,1.0,110,Other,1.000000,110,...,Other,1.000000,110,Other,1.000000,Other,1.00000,Trypanosoma_cruzi_Dm28c_2018,Mucin-associated surface protein (MASP) [Sourc...,Mucin-associated surface protein (MASP)
C4B63_1g1003-t42_1-p1,MMTCRLLCALLVLALCCCPSVSAAGTQEAVKEPSKSTTTNDTTKAQ...,C4B63_1g1003-t42_1-p1 | transcript=C4B63_1g100...,224,110,Other,1.0,110,Other,1.000000,110,...,Other,1.000000,110,Other,1.000000,Other,1.00000,Trypanosoma_cruzi_Dm28c_2018,Putative mucin TcMUCII [Source:UniProtKB/TrEMB...,mucin TcMUCII
C4B63_1g1004-t42_1-p1,MAMMMTGRVLLVCALCVLWCGAGGVYARYLDNNPVGGCMASGGFGA...,C4B63_1g1004-t42_1-p1 | transcript=C4B63_1g100...,420,110,Other,1.0,110,Other,1.000000,110,...,Other,1.000000,110,Other,1.000000,Other,1.00000,Trypanosoma_cruzi_Dm28c_2018,Mucin-associated surface protein (MASP) [Sourc...,Mucin-associated surface protein (MASP)
C4B63_1g1025-t42_1-p1,MAMMMTGRVLLVCALCVLWCGAGEGGAEMNDETLVGSQHLTGVGEK...,C4B63_1g1025-t42_1-p1 | transcript=C4B63_1g102...,479,110,Other,1.0,110,Other,1.000000,110,...,Other,1.000000,110,Other,1.000000,Other,1.00000,Trypanosoma_cruzi_Dm28c_2018,Mucin-associated surface protein (MASP) [Sourc...,Mucin-associated surface protein (MASP)
C4B63_1g1031-t42_1-p1,MVLKRLLWVWTPHPHQYAYRSMRTTTMQLAHLIHEVEHNRNHYFQV...,C4B63_1g1031-t42_1-p1 | transcript=C4B63_1g103...,761,110,Other,1.0,110,Other,0.999886,110,...,Other,0.999874,110,Other,0.999871,Other,0.99992,Trypanosoma_cruzi_Dm28c_2018,L1Tc protein fragment,L1Tc protein fragment


In [26]:
list_df_1000[list_df_1000['vote'].str.startswith('GT')].groupby('organism')['vote'].value_counts()

organism                      vote 
Leishmania_donovani_BPK282A1  GT108    7
                              GT67     6
                              GT4      5
                              GT40     4
                              GT66     4
                                      ..
Trypanosoma_cruzi_Dm28c_2018  GT60     1
                              GT61     1
                              GT71     1
                              GT76     1
                              GT77     1
Name: vote, Length: 69, dtype: int64

In [27]:
#list_df_1000[list_df_1000['organism']=='Plasmodium_falciparum_3D7']
list_df_1000[(list_df_1000['organism']=='Plasmodium_falciparum_3D7')&
            (list_df_1000['vote'].str.startswith('GT')) & (~list_df_1000['transcript_product'].str.contains('glycosyltransferase')) ][
   ['vote','vote_mean','organism','gene_product','transcript_product','len_seq']].sort_values(by='vote_mean').tail(50)

,vote,vote_mean,organism,gene_product,transcript_product,len_seq
PF3D7_1354600.1-p1,GT2,0.835344,Plasmodium_falciparum_3D7,"60S ribosomal protein L7-2, putative","60S ribosomal protein L7-2, putative",252
PF3D7_0107400.1-p1,GT2,0.867826,Plasmodium_falciparum_3D7,"ATP synthase-associated protein, putative","ATP synthase-associated protein, putative",153
PF3D7_1363300.1-p1,GT2,0.935202,Plasmodium_falciparum_3D7,"50S ribosomal protein L9, mitochondrial, putative","50S ribosomal protein L9, mitochondrial, putative",250
PF3D7_1018400.1-p1,GT2,0.988618,Plasmodium_falciparum_3D7,"conserved Plasmodium protein, unknown function","conserved Plasmodium protein, unknown function",199
PF3D7_1341600.1-p1,GT22,0.999995,Plasmodium_falciparum_3D7,GPI mannosyltransferase 3,GPI mannosyltransferase 3,786
PF3D7_0211600.1-p1,GT1,1.000000,Plasmodium_falciparum_3D7,UDP-N-acetylglucosamine transferase subunit AL...,UDP-N-acetylglucosamine transferase subunit AL...,209
PF3D7_0806400.1-p1,GT1,1.000000,Plasmodium_falciparum_3D7,UDP-N-acetylglucosamine transferase subunit AL...,UDP-N-acetylglucosamine transferase subunit AL...,177
PF3D7_0909200.1-p1,GT68,1.000000,Plasmodium_falciparum_3D7,GDP-fucose protein O-fucosyltransferase 2,GDP-fucose protein O-fucosyltransferase 2,469
PF3D7_1032400.1-p1,GT4,1.000000,Plasmodium_falciparum_3D7,phosphatidylinositol N-acetylglucosaminyltrans...,phosphatidylinositol N-acetylglucosaminyltrans...,461
PF3D7_1141600.1-p1,GT2,1.000000,Plasmodium_falciparum_3D7,dolichol-phosphate mannosyltransferase,dolichol-phosphate mannosyltransferase,259


In [28]:
#list_df_1000[list_df_1000['organism']=='Plasmodium_falciparum_3D7']
list_df_1000[(list_df_1000['organism']=='Trypanosoma_cruzi_Dm28c_2018')&
            (list_df_1000['vote'].str.startswith('GT')) & (~list_df_1000['gene_product'].str.contains('transferase')) ][
   ['vote','vote_mean','organism','gene_product','transcript_product','len_seq']].sort_values(by='vote_mean').tail(5)

,vote,vote_mean,organism,gene_product,transcript_product,len_seq
C4B63_297g4-t42_1-p1,GT31,0.998431,Trypanosoma_cruzi_Dm28c_2018,conserved hypothetical protein,conserved hypothetical protein,535
C4B63_17g58-t42_1-p1,GT31,0.998590,Trypanosoma_cruzi_Dm28c_2018,conserved hypothetical protein,conserved hypothetical protein,535
C4B63_34g116-t42_1-p1,GT4,0.998894,Trypanosoma_cruzi_Dm28c_2018,Glycos_transf_1 domain-containing protein [Sou...,mannosyltransferase-like protein,859
C4B63_13g37-t42_1-p1,GT4,1.000000,Trypanosoma_cruzi_Dm28c_2018,GlcNAc-PI synthesis protein [Source:UniProtKB/...,GlcNAc-PI synthesis protein,455
C4B63_40g41-t42_1-p1,GT4,1.000000,Trypanosoma_cruzi_Dm28c_2018,Glyco_trans_4-like_N domain-containing protein...,glycosyl transferase-like,552


In [29]:
#list_df_1000[list_df_1000['organism']=='Plasmodium_falciparum_3D7']
list_df_1000[(list_df_1000['organism']=='Leishmania_donovani_BPK282A1')&
            (list_df_1000['vote'].str.startswith('GT')) & (~list_df_1000['gene_product'].str.contains('transferase')) ][
   ['vote','vote_mean','organism','gene_product','transcript_product','len_seq']].sort_values(by='vote_mean').tail(10)

,vote,vote_mean,organism,gene_product,transcript_product,len_seq
LdBPK_101410.1.1-p1,GT108,0.999999,Leishmania_donovani_BPK282A1,"Protein of unknown function (DUF1861), putative","Protein of unknown function (DUF1861), putative",313
LdBPK_101380.1.1-p1,GT108,1.000000,Leishmania_donovani_BPK282A1,"Protein of unknown function (DUF1861), putative","Protein of unknown function (DUF1861), putative",321
LdBPK_101430.1.1-p1,GT108,1.000000,Leishmania_donovani_BPK282A1,"Protein of unknown function (DUF1861), putative","Protein of unknown function (DUF1861), putative",311
LdBPK_101420.1.1-p1,GT108,1.000000,Leishmania_donovani_BPK282A1,"Protein of unknown function (DUF1861), putative","Protein of unknown function (DUF1861), putative",316
LdBPK_101400.1.1-p1,GT108,1.000000,Leishmania_donovani_BPK282A1,"Protein of unknown function (DUF1861), putative","Protein of unknown function (DUF1861), putative",315
LdBPK_101390.1.1-p1,GT108,1.000000,Leishmania_donovani_BPK282A1,"Protein of unknown function (DUF1861), putative","Protein of unknown function (DUF1861), putative",318
LdBPK_331770.1.1-p1,GT4,1.000000,Leishmania_donovani_BPK282A1,GlcNAc-PI synthesis protein,GlcNAc-PI synthesis protein,454
LdBPK_101370.1.1-p1,GT108,1.000000,Leishmania_donovani_BPK282A1,"Protein of unknown function (DUF1861), putative","Protein of unknown function (DUF1861), putative",328
LdBPK_360240.1.1-p1,GT2,1.000000,Leishmania_donovani_BPK282A1,dolicholphosphate-mannose synthase,dolicholphosphate-mannose synthase,262
LdBPK_160960.1.1-p1,GT4,1.000000,Leishmania_donovani_BPK282A1,sucrose-phosphate synthase-like protein,sucrose-phosphate synthase-like protein,464


In [30]:
#list_df_1000[list_df_1000['organism']=='Plasmodium_falciparum_3D7']
list_df_1000[(list_df_1000['organism']=='Trypanosoma_brucei_brucei_TREU927')&
            (list_df_1000['vote'].str.startswith('GT')) & (~list_df_1000['gene_product'].str.contains('transferase')) ][
   ['vote','vote_mean','organism','gene_product','transcript_product','len_seq']].sort_values(by='vote_mean').tail(5)

,vote,vote_mean,organism,gene_product,transcript_product,len_seq
Tb927.11.20410:mRNA-p1,GT67,0.999999,Trypanosoma_brucei_brucei_TREU927,hypothetical protein,hypothetical protein,180
Tb09.v4.0080:mRNA-p1,GT67,1.000000,Trypanosoma_brucei_brucei_TREU927,"hypothetical protein, conserved","hypothetical protein, conserved",327
Tb11.v5.0765.1-p1,GT4,1.000000,Trypanosoma_brucei_brucei_TREU927,N-acetylglucosaminyl-phosphatidylinositol bios...,N-acetylglucosaminyl-phosphatidylinositol bios...,428
Tb927.10.4610:mRNA-p1,GT2,1.000000,Trypanosoma_brucei_brucei_TREU927,"dolicholphosphate-mannose synthase, putative","dolicholphosphate-mannose synthase, putative",259
Tb927.2.1780:mRNA-p1,GT4,1.000000,Trypanosoma_brucei_brucei_TREU927,GlcNAc-PI synthesis protein,GlcNAc-PI synthesis protein,455


In [31]:
list_df_1000[list_df_1000['vote']!='Other']['vote'].value_counts()

GT67     143
GT2       43
GT4       19
GT11      13
GT1       13
GT66      13
GT40      13
GT22      10
GT31       8
GT108      7
GT47       7
GT25       6
GT76       5
GT60       5
GT8        5
GT33       4
GT50       4
GT58       3
GT105      2
GT49       2
GT71       2
GT77       2
GT61       2
GT90       2
GT51       1
GT13       1
GT10       1
GT26       1
GT68       1
Name: vote, dtype: int64

In [32]:
list_df_1000.to_pickle('list_df_1000.pkl',compression='gzip')

In [183]:

list_df_500['vote']=list_df_500[['model_0_family_1','model_1_family_1','model_2_family_1','model_3_family_1','model_4_family_1']].apply(select,axis=1)

/tmp/446601.1.all.q/ipykernel_46518/3094935499.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  list_df_500['vote']=list_df_500[['model_0_family_1','model_1_family_1','model_2_family_1','model_3_family_1','model_4_family_1']].apply(select,axis=1)


In [184]:
list_df_500['vote_mean']=list_df_500[['model_0_prob_1','model_1_prob_1','model_2_prob_1','model_3_prob_1','model_4_prob_1']].mean(axis=1)

/tmp/446601.1.all.q/ipykernel_46518/174577039.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  list_df_500['vote_mean']=list_df_500[['model_0_prob_1','model_1_prob_1','model_2_prob_1','model_3_prob_1','model_4_prob_1']].mean(axis=1)


In [165]:
list_df_500[list_df_500['vote']!='Other']

,seq,desc,len_seq,model_0_labels_1,model_0_family_1,model_0_prob_1,model_1_labels_1,model_1_family_1,model_1_prob_1,model_2_labels_1,...,model_3_labels_1,model_3_family_1,model_3_prob_1,model_4_labels_1,model_4_family_1,model_4_prob_1,vote,organism,gene_product,transcript_product
C4B63_2g305-t42_1-p1,MSTVEELRQSGNAAFSAGRYQEALTFYEQALALDAENVDLLNNAAA...,C4B63_2g305-t42_1-p1 | transcript=C4B63_2g305-...,352,110,Other,0.999862,110,Other,0.999462,7,...,7,GT105,0.992433,7,GT105,0.960168,GT105,Trypanosoma_cruzi_Dm28c_2018,Hsp70-Hsp90 organizing protein 3,Hsp70-Hsp90 organizing protein 3
C4B63_2g337-t42_1-p1,MLLAGTIAAVLFLLSLAAAVGMTARRGKRRFVRGTVGFLHPSAAAG...,C4B63_2g337-t42_1-p1 | transcript=C4B63_2g337-...,473,47,GT4,1.000000,47,GT4,1.000000,47,...,47,GT4,1.000000,47,GT4,1.000000,GT4,Trypanosoma_cruzi_Dm28c_2018,"GDP-Man:Man(3)GlcNAc(2)-PP-Dol alpha-1,2-manno...","alpha-1,2-mannosyltransferase"
C4B63_2g656-t42_1-p1,MWPDLALRLFPTAMYIAKDDDDIFLRVPLFVAILRLLPRRGIYMGY...,C4B63_2g656-t42_1-p1 | transcript=C4B63_2g656-...,193,76,GT67,1.000000,39,GT31,0.998626,76,...,76,GT67,1.000000,76,GT67,1.000000,GT67,Trypanosoma_cruzi_Dm28c_2018,Hexosyltransferase [Source:UniProtKB/TrEMBL;Ac...,UDP-Gal or UDP-GlcNAc-dependent glycosyltransf...
C4B63_3g1018-t42_1-p1,MPPSMHNKLATSIALPRDAVINGLRRVCNTIGIPHSHNRAPPIQHR...,C4B63_3g1018-t42_1-p1 | transcript=C4B63_3g101...,198,110,Other,0.801809,0,GT1,0.800407,0,...,0,GT1,0.653030,110,Other,0.846167,GT1,Trypanosoma_cruzi_Dm28c_2018,conserved hypothetical protein,conserved hypothetical protein
C4B63_3g1128-t42_1-p1,MLVLYVIGRHPSHGYDYSAAMLEEAAQWNDVVALPMNEGLVSPTKK...,C4B63_3g1128-t42_1-p1 | transcript=C4B63_3g112...,259,76,GT67,1.000000,76,GT67,1.000000,76,...,76,GT67,1.000000,76,GT67,1.000000,GT67,Trypanosoma_cruzi_Dm28c_2018,Hexosyltransferase [Source:UniProtKB/TrEMBL;Ac...,UDP-Gal or UDP-GlcNAc-dependent glycosyltransf...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PF3D7_0934200.1-p1,MNYNLIYNKINTNYVIDPYVSMLYIKRAYKYLRLLKKNNGNVLVLG...,PF3D7_0934200.1-p1 | transcript=PF3D7_0934200....,145,47,GT4,0.995655,110,Other,0.984564,47,...,110,Other,0.913042,47,GT4,0.933034,GT4,Plasmodium_falciparum_3D7,"conserved protein, unknown function","conserved protein, unknown function"
PF3D7_1032400.1-p1,MESAVSECNIYKKEKDKNIIYKQERKCCICMVSDFFYPNLGGIETH...,PF3D7_1032400.1-p1 | transcript=PF3D7_1032400....,461,47,GT4,1.000000,47,GT4,1.000000,47,...,47,GT4,1.000000,47,GT4,1.000000,GT4,Plasmodium_falciparum_3D7,phosphatidylinositol N-acetylglucosaminyltrans...,phosphatidylinositol N-acetylglucosaminyltrans...
PF3D7_1141600.1-p1,MVIRFFLFVITLLGLCINMVCCNFKYSIILPTYNEKENLPYLIYMI...,PF3D7_1141600.1-p1 | transcript=PF3D7_1141600....,259,26,GT2,1.000000,26,GT2,1.000000,26,...,26,GT2,1.000000,26,GT2,1.000000,GT2,Plasmodium_falciparum_3D7,dolichol-phosphate mannosyltransferase,dolichol-phosphate mannosyltransferase
PF3D7_1210900.1-p1,MGHTHKEYKNNEKSAIFFEWLIFFVGIIIRIIIYYYGRWQDKNFNV...,PF3D7_1210900.1-p1 | transcript=PF3D7_1210900....,441,58,GT50,1.000000,58,GT50,1.000000,58,...,58,GT50,1.000000,58,GT50,1.000000,GT50,Plasmodium_falciparum_3D7,GPI mannosyltransferase 1,GPI mannosyltransferase 1


In [1]:
import pandas as pd
list_df_1000 = pd.read_pickle('list_df_1000.pkl',compression='gzip')

In [3]:
list_df_1000.loc['Tb927.5.2760:mRNA-p1']

seq                   MFARVKRVRQKRSMFLTVIALIIGIVVLRKLIHYSVQKHQEMSDEY...
desc                  Tb927.5.2760:mRNA-p1 | transcript=Tb927.5.2760...
len_seq                                                             384
model_0_labels_1                                                     76
model_0_family_1                                                   GT67
model_0_prob_1                                                      1.0
model_1_labels_1                                                     76
model_1_family_1                                                   GT67
model_1_prob_1                                                      1.0
model_2_labels_1                                                     76
model_2_family_1                                                   GT67
model_2_prob_1                                                      1.0
model_3_labels_1                                                     76
model_3_family_1                                                

In [4]:
list_df_1000.loc['Tb927.9.3600:mRNA-p1']

seq                   MSNKAFLTTNLVGGLGNQLFLVANLLATARRNGLTAVLPRMSFSSS...
desc                  Tb927.9.3600:mRNA-p1 | transcript=Tb927.9.3600...
len_seq                                                             308
model_0_labels_1                                                     12
model_0_family_1                                                   GT11
model_0_prob_1                                                      1.0
model_1_labels_1                                                     12
model_1_family_1                                                   GT11
model_1_prob_1                                                      1.0
model_2_labels_1                                                     12
model_2_family_1                                                   GT11
model_2_prob_1                                                      1.0
model_3_labels_1                                                     12
model_3_family_1                                                

In [6]:
list_df_1000.loc['Tb927.11.20410:mRNA-p1']

seq                   MKADDDIFMRVPLYLKYLELLPREKLNMGRAVGYPKEFSDEITWHT...
desc                  Tb927.11.20410:mRNA-p1 | transcript=Tb927.11.2...
len_seq                                                             180
model_0_labels_1                                                     76
model_0_family_1                                                   GT67
model_0_prob_1                                                      1.0
model_1_labels_1                                                     76
model_1_family_1                                                   GT67
model_1_prob_1                                                      1.0
model_2_labels_1                                                     76
model_2_family_1                                                   GT67
model_2_prob_1                                                 0.999995
model_3_labels_1                                                     76
model_3_family_1                                                

In [3]:
!pip freeze > requirements.txt

In [2]:
#list_df_1000[list_df_1000['organism']=='Plasmodium_falciparum_3D7']
list_df_1000[(list_df_1000['organism']=='Trypanosoma_brucei_brucei_TREU927')&
            (list_df_1000['vote'].str.startswith('GT')) & (~list_df_1000['gene_product'].str.contains('transferase')) ][
   ['vote','vote_mean','organism','gene_product','transcript_product','len_seq']].sort_values(by='vote_mean').tail(10)

,vote,vote_mean,organism,gene_product,transcript_product,len_seq
Tb11.0740:mRNA-p1,GT67,0.996737,Trypanosoma_brucei_brucei_TREU927,hypothetical protein,hypothetical protein,183
Tb11.1160:mRNA-p1,GT67,0.998178,Trypanosoma_brucei_brucei_TREU927,hypothetical protein,hypothetical protein,177
Tb927.11.18780.1-p1,GT67,0.998178,Trypanosoma_brucei_brucei_TREU927,"hypothetical protein, conserved","hypothetical protein, conserved",177
Tb11.0730:mRNA-p1,GT67,0.999785,Trypanosoma_brucei_brucei_TREU927,hypothetical protein,hypothetical protein,171
Tb10.v4.0188:mRNA-p1,GT67,0.999980,Trypanosoma_brucei_brucei_TREU927,hypothetical protein,hypothetical protein,254
Tb927.11.20410:mRNA-p1,GT67,0.999999,Trypanosoma_brucei_brucei_TREU927,hypothetical protein,hypothetical protein,180
Tb09.v4.0080:mRNA-p1,GT67,1.000000,Trypanosoma_brucei_brucei_TREU927,"hypothetical protein, conserved","hypothetical protein, conserved",327
Tb11.v5.0765.1-p1,GT4,1.000000,Trypanosoma_brucei_brucei_TREU927,N-acetylglucosaminyl-phosphatidylinositol bios...,N-acetylglucosaminyl-phosphatidylinositol bios...,428
Tb927.10.4610:mRNA-p1,GT2,1.000000,Trypanosoma_brucei_brucei_TREU927,"dolicholphosphate-mannose synthase, putative","dolicholphosphate-mannose synthase, putative",259
Tb927.2.1780:mRNA-p1,GT4,1.000000,Trypanosoma_brucei_brucei_TREU927,GlcNAc-PI synthesis protein,GlcNAc-PI synthesis protein,455


In [13]:
list_df_1000[(list_df_1000['organism']=='Plasmodium_falciparum_3D7')&
            (list_df_1000['vote'].str.startswith('GT')) ][['vote','vote_mean','organism','gene_product','transcript_product','len_seq']].sort_values(by='vote_mean')

,vote,vote_mean,organism,gene_product,transcript_product,len_seq
PF3D7_1354600.1-p1,GT2,0.835344,Plasmodium_falciparum_3D7,"60S ribosomal protein L7-2, putative","60S ribosomal protein L7-2, putative",252
PF3D7_0107400.1-p1,GT2,0.867826,Plasmodium_falciparum_3D7,"ATP synthase-associated protein, putative","ATP synthase-associated protein, putative",153
PF3D7_1363300.1-p1,GT2,0.935202,Plasmodium_falciparum_3D7,"50S ribosomal protein L9, mitochondrial, putative","50S ribosomal protein L9, mitochondrial, putative",250
PF3D7_1018400.1-p1,GT2,0.988618,Plasmodium_falciparum_3D7,"conserved Plasmodium protein, unknown function","conserved Plasmodium protein, unknown function",199
PF3D7_1341600.1-p1,GT22,0.999995,Plasmodium_falciparum_3D7,GPI mannosyltransferase 3,GPI mannosyltransferase 3,786
PF3D7_1116600.1-p1,GT66,1.000000,Plasmodium_falciparum_3D7,dolichyl-diphosphooligosaccharide--protein gly...,dolichyl-diphosphooligosaccharide--protein gly...,856
PF3D7_0211600.1-p1,GT1,1.000000,Plasmodium_falciparum_3D7,UDP-N-acetylglucosamine transferase subunit AL...,UDP-N-acetylglucosamine transferase subunit AL...,209
PF3D7_0806400.1-p1,GT1,1.000000,Plasmodium_falciparum_3D7,UDP-N-acetylglucosamine transferase subunit AL...,UDP-N-acetylglucosamine transferase subunit AL...,177
PF3D7_0909200.1-p1,GT68,1.000000,Plasmodium_falciparum_3D7,GDP-fucose protein O-fucosyltransferase 2,GDP-fucose protein O-fucosyltransferase 2,469
PF3D7_1032400.1-p1,GT4,1.000000,Plasmodium_falciparum_3D7,phosphatidylinositol N-acetylglucosaminyltrans...,phosphatidylinositol N-acetylglucosaminyltrans...,461


In [4]:
list_df_1000[(list_df_1000['organism']=='Trypanosoma_brucei_brucei_TREU927')&
            (list_df_1000['vote'].str.startswith('GT')) ][
   ['vote','vote_mean','organism','gene_product','transcript_product','len_seq']].sort_values(by='vote_mean').to_csv('all_gt_tbrucei.csv')

In [11]:
list_df_1000[(list_df_1000['organism']=='Trypanosoma_brucei_brucei_TREU927')&
            (list_df_1000['vote'].str.startswith('GT')) ][
   ['vote','vote_mean','organism','gene_product','transcript_product','len_seq']].sort_values(by='vote_mean').head(50)

,vote,vote_mean,organism,gene_product,transcript_product,len_seq
Tb927.7.6940:mRNA-p1,GT47,0.652536,Trypanosoma_brucei_brucei_TREU927,"glutaminyl cyclase, putative","glutaminyl cyclase, putative",747
Tb927.10.8010:mRNA-p1,GT2,0.753464,Trypanosoma_brucei_brucei_TREU927,"hypothetical protein, conserved","hypothetical protein, conserved",244
Tb927.11.2020:mRNA-p1,GT67,0.805500,Trypanosoma_brucei_brucei_TREU927,"hypothetical protein, conserved","hypothetical protein, conserved",258
Tb927.11.1520:mRNA-p1,GT11,0.835294,Trypanosoma_brucei_brucei_TREU927,expression site-associated gene 3 (ESAG3) prot...,expression site-associated gene 3 (ESAG3) prot...,390
Tb927.3.5410:mRNA-p1,GT2,0.836922,Trypanosoma_brucei_brucei_TREU927,"hypothetical protein, conserved","hypothetical protein, conserved",203
Tb05.5K5.240:mRNA-p1,GT11,0.864432,Trypanosoma_brucei_brucei_TREU927,expression site-associated gene 3 (ESAG3) prot...,expression site-associated gene 3 (ESAG3) prot...,395
Tb927.5.4600:mRNA-p1,GT11,0.864432,Trypanosoma_brucei_brucei_TREU927,expression site-associated gene 3 (ESAG3) prot...,expression site-associated gene 3 (ESAG3) prot...,395
Tb11.v5.0367.1-p1,GT11,0.864432,Trypanosoma_brucei_brucei_TREU927,expression site-associated gene (ESAG) protein...,expression site-associated gene (ESAG) protein...,395
Tb927.10.9490:mRNA-p1,GT11,0.872436,Trypanosoma_brucei_brucei_TREU927,expression site-associated gene 3 (ESAG3)-like...,expression site-associated gene 3 (ESAG3)-like...,392
Tb927.9.15940:mRNA-p1,GT11,0.884444,Trypanosoma_brucei_brucei_TREU927,expression site-associated gene 3 (ESAG3) prot...,expression site-associated gene 3 (ESAG3) prot...,395


In [12]:
list_df_1000[(list_df_1000['organism']=='Leishmania_donovani_BPK282A1')&
            (list_df_1000['vote'].str.startswith('GT')) ][
   ['vote','vote_mean','organism','gene_product','transcript_product','len_seq']].sort_values(by='vote_mean').head(50)

,vote,vote_mean,organism,gene_product,transcript_product,len_seq
LdBPK_150020.1.1-p1,GT77,0.610251,Leishmania_donovani_BPK282A1,"hypothetical protein, conserved","hypothetical protein, conserved",620
LdBPK_212180.1.1-p1,GT8,0.704169,Leishmania_donovani_BPK282A1,"GPI transamidase component GAA1, putative","GPI transamidase component GAA1, putative",464
LdBPK_061340.1.1-p1,GT1,0.721046,Leishmania_donovani_BPK282A1,protoporphyrinogen oxidase-like protein,protoporphyrinogen oxidase-like protein,231
LdBPK_280700.1.1-p1,GT2,0.801419,Leishmania_donovani_BPK282A1,"hypothetical protein, conserved","hypothetical protein, conserved",340
LdBPK_333010.1.1-p1,GT8,0.808456,Leishmania_donovani_BPK282A1,"hypothetical protein, conserved","hypothetical protein, conserved",484
LdBPK_321600.1.1-p1,GT49,0.841815,Leishmania_donovani_BPK282A1,"hypothetical protein, conserved","hypothetical protein, conserved",587
LdBPK_361260.1.1-p1,GT22,0.875937,Leishmania_donovani_BPK282A1,"GPI anchor biosynthesis protein, putative","GPI anchor biosynthesis protein, putative",626
LdBPK_260520.1.1-p1,GT40,0.885025,Leishmania_donovani_BPK282A1,galactofuranosyltransferase lpg1-like protein,galactofuranosyltransferase lpg1-like protein,538
LdBPK_200170.1.1-p1,GT105,0.921604,Leishmania_donovani_BPK282A1,"Domain of unknown function (DUF1736), putative","Domain of unknown function (DUF1736), putative",908
LdBPK_020170.1.1-p1,GT67,0.978170,Leishmania_donovani_BPK282A1,"phosphoglycan beta 1,3 galactosyltransferase","phosphoglycan beta 1,3 galactosyltransferase",361


In [166]:
list_df_500[list_df_500['vote']!='Other']['vote'].value_counts()

GT67     143
GT2       38
GT4       15
GT40      10
GT1        9
GT31       9
GT25       7
GT108      7
GT66       7
GT11       6
GT50       4
GT58       3
GT47       3
GT90       3
GT68       2
GT57       2
GT22       2
GT8        2
GT91       1
GT49       1
GT51       1
GT105      1
GT61       1
GT73       1
GT71       1
GT14       1
GT13       1
GT60       1
GT64       1
GT10       1
GT76       1
Name: vote, dtype: int64

In [167]:
list_df_500[list_df_500['vote']!='Other'].to_pickle('predict_500.pkl',compression='gzip')

In [169]:
list_df_500['organism']=list_df_500['desc'].apply(get_desc,args=('organism',))
list_df_500['gene_product']=list_df_500['desc'].apply(get_desc,args=('gene_product',))
list_df_500['transcript_product']=list_df_500['desc'].apply(get_desc,args=('transcript_product',))

/tmp/446601.1.all.q/ipykernel_46518/2256009241.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  list_df_500['organism']=list_df_500['desc'].apply(get_desc,args=('organism',))
/tmp/446601.1.all.q/ipykernel_46518/2256009241.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  list_df_500['gene_product']=list_df_500['desc'].apply(get_desc,args=('gene_product',))
/tmp/446601.1.all.q/ipykernel_46518/2256009241.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [170]:
list_df_500.head()

,seq,desc,len_seq,model_0_labels_1,model_0_family_1,model_0_prob_1,model_1_labels_1,model_1_family_1,model_1_prob_1,model_2_labels_1,...,model_3_labels_1,model_3_family_1,model_3_prob_1,model_4_labels_1,model_4_family_1,model_4_prob_1,vote,organism,gene_product,transcript_product
C4B63_1g1002-t42_1-p1,MAMMMTGRVLLVCALCVLWCGTSGRCDDAKQKELLDQRSVSPLGGE...,C4B63_1g1002-t42_1-p1 | transcript=C4B63_1g100...,380,110,Other,1.0,110,Other,1.0,110,...,110,Other,1.0,110,Other,1.0,Other,Trypanosoma_cruzi_Dm28c_2018,Mucin-associated surface protein (MASP) [Sourc...,Mucin-associated surface protein (MASP)
C4B63_1g1003-t42_1-p1,MMTCRLLCALLVLALCCCPSVSAAGTQEAVKEPSKSTTTNDTTKAQ...,C4B63_1g1003-t42_1-p1 | transcript=C4B63_1g100...,224,110,Other,1.0,110,Other,1.0,110,...,110,Other,1.0,110,Other,1.0,Other,Trypanosoma_cruzi_Dm28c_2018,Putative mucin TcMUCII [Source:UniProtKB/TrEMB...,mucin TcMUCII
C4B63_1g1004-t42_1-p1,MAMMMTGRVLLVCALCVLWCGAGGVYARYLDNNPVGGCMASGGFGA...,C4B63_1g1004-t42_1-p1 | transcript=C4B63_1g100...,420,110,Other,1.0,110,Other,1.0,110,...,110,Other,1.0,110,Other,1.0,Other,Trypanosoma_cruzi_Dm28c_2018,Mucin-associated surface protein (MASP) [Sourc...,Mucin-associated surface protein (MASP)
C4B63_1g1025-t42_1-p1,MAMMMTGRVLLVCALCVLWCGAGEGGAEMNDETLVGSQHLTGVGEK...,C4B63_1g1025-t42_1-p1 | transcript=C4B63_1g102...,479,110,Other,1.0,110,Other,1.0,110,...,110,Other,1.0,110,Other,1.0,Other,Trypanosoma_cruzi_Dm28c_2018,Mucin-associated surface protein (MASP) [Sourc...,Mucin-associated surface protein (MASP)
C4B63_1g1037-t42_1-p1,MAMMTGRVLLVCALCVLWCGAGCGFAKEKAVQDVQTDVSGDDERHK...,C4B63_1g1037-t42_1-p1 | transcript=C4B63_1g103...,412,110,Other,1.0,110,Other,1.0,110,...,110,Other,1.0,110,Other,1.0,Other,Trypanosoma_cruzi_Dm28c_2018,Mucin-associated surface protein (MASP) [Sourc...,Mucin-associated surface protein (MASP)


In [171]:
list_df_500[(list_df_500['vote']!='Other') & (list_df_500['gene_product'].str.contains('hypothetical'))].head(50)

,seq,desc,len_seq,model_0_labels_1,model_0_family_1,model_0_prob_1,model_1_labels_1,model_1_family_1,model_1_prob_1,model_2_labels_1,...,model_3_labels_1,model_3_family_1,model_3_prob_1,model_4_labels_1,model_4_family_1,model_4_prob_1,vote,organism,gene_product,transcript_product
C4B63_3g1018-t42_1-p1,MPPSMHNKLATSIALPRDAVINGLRRVCNTIGIPHSHNRAPPIQHR...,C4B63_3g1018-t42_1-p1 | transcript=C4B63_3g101...,198,110,Other,0.801809,0,GT1,0.800407,0,...,0,GT1,0.653030,110,Other,0.846167,GT1,Trypanosoma_cruzi_Dm28c_2018,conserved hypothetical protein,conserved hypothetical protein
C4B63_9g445-t42_1-p1,MSLCYKSRGLLFKCVFVLMAVYLCFFVPWPEHNANKQGTRSTQIHQ...,C4B63_9g445-t42_1-p1 | transcript=C4B63_9g445-...,473,39,GT31,0.983454,39,GT31,0.999442,39,...,110,Other,0.770376,39,GT31,0.836103,GT31,Trypanosoma_cruzi_Dm28c_2018,conserved hypothetical protein,conserved hypothetical protein
C4B63_26g1224c-t42_1-p1,MHVEAEEEVEGGGVFSTMHSTLASTEGDDFIKRDLEVLDALIKEGY...,C4B63_26g1224c-t42_1-p1 | transcript=C4B63_26g...,121,26,GT2,0.999914,26,GT2,0.999659,26,...,26,GT2,0.999391,26,GT2,0.996788,GT2,Trypanosoma_cruzi_Dm28c_2018,conserved hypothetical protein,conserved hypothetical protein
C4B63_27g241-t42_1-p1,MSAVAVAVMETGISSVCTMGTPECDDELYLFLRCLRVHHPDIPVIV...,C4B63_27g241-t42_1-p1 | transcript=C4B63_27g24...,334,26,GT2,0.880683,110,Other,0.695760,26,...,26,GT2,0.954944,110,Other,0.646629,GT2,Trypanosoma_cruzi_Dm28c_2018,conserved hypothetical protein,conserved hypothetical protein
C4B63_28g66-t42_1-p1,MSVNFCFATGLFILFLCVLLTAFEPWLRCDDVFGLDGEDHISVLRN...,C4B63_28g66-t42_1-p1 | transcript=C4B63_28g66-...,444,0,GT1,0.828167,0,GT1,0.999216,0,...,110,Other,0.982469,0,GT1,0.989722,GT1,Trypanosoma_cruzi_Dm28c_2018,conserved hypothetical protein,conserved hypothetical protein
C4B63_28g83-t42_1-p1,MRRRDMHRRFLWRKKAVEYLFAIFCFLTFQILFGLVVMRNSTPCVG...,C4B63_28g83-t42_1-p1 | transcript=C4B63_28g83-...,381,19,GT13,1.000000,62,GT54,0.699347,19,...,11,GT109,0.996436,19,GT13,0.467081,GT13,Trypanosoma_cruzi_Dm28c_2018,conserved hypothetical protein,conserved hypothetical protein
C4B63_30g179-t42_1-p1,MSFPLPKKYYLLVHDAARKVLGTDAEISAFVSSKRNFQCDMLINAS...,C4B63_30g179-t42_1-p1 | transcript=C4B63_30g17...,273,20,GT14,0.988932,20,GT14,0.900221,110,...,110,Other,0.999729,20,GT14,0.957455,GT14,Trypanosoma_cruzi_Dm28c_2018,conserved hypothetical protein,conserved hypothetical protein
C4B63_48g158-t42_1-p1,MTNPLRVAALIYLFFFFFFFLTCGEAAVLEELVLPYCVRGWRWKDV...,C4B63_48g158-t42_1-p1 | transcript=C4B63_48g15...,478,70,GT61,1.000000,70,GT61,0.999968,70,...,70,GT61,0.985519,70,GT61,0.998210,GT61,Trypanosoma_cruzi_Dm28c_2018,conserved hypothetical protein,conserved hypothetical protein
C4B63_63g38-t42_1-p1,MTAEQLVTLKGVLLTDLWNGPLGAVSRTRERLGHIDFAMVVPPLRA...,C4B63_63g38-t42_1-p1 | transcript=C4B63_63g38-...,244,26,GT2,0.930842,26,GT2,0.954143,26,...,110,Other,0.937329,110,Other,0.617034,GT2,Trypanosoma_cruzi_Dm28c_2018,conserved hypothetical protein,conserved hypothetical protein
C4B63_75g103-t42_1-p1,MPLFSLRSSTWGIFIILCGMESTRPLAYRRVVVSGSQDPVEYFVRA...,C4B63_75g103-t42_1-p1 | transcript=C4B63_75g10...,216,39,GT31,0.999999,39,GT31,0.999992,39,...,39,GT31,0.580470,39,GT31,0.999753,GT31,Trypanosoma_cruzi_Dm28c_2018,conserved hypothetical protein,conserved hypothetical protein


In [185]:
list_df_500[list_df_500['vote'].str.startswith('GT')].groupby('organism')['vote'].value_counts()

organism                           vote 
Leishmania_donovani_BPK282A1       GT108      7
                                   GT2        4
                                   GT25       3
                                   GT4        3
                                   GT66       3
                                   GT67       3
                                   GT1        2
                                   GT40       2
                                   GT11       1
                                   GT22       1
                                   GT49       1
                                   GT50       1
                                   GT58       1
                                   GT8        1
                                   GT90       1
Plasmodium_falciparum_3D7          GT2        3
                                   GT1        2
                                   GT4        2
                                   GT50       1
                                   GT68       1

In [125]:
list_df_500[list_df_500['organism']=='Plasmodium_falciparum_3D7']

,seq,desc,len_seq,model_0_labels_1,model_0_family_1,model_0_prob_1,model_1_labels_1,model_1_family_1,model_1_prob_1,model_2_labels_1,...,model_3_labels_1,model_3_family_1,model_3_prob_1,model_4_labels_1,model_4_family_1,model_4_prob_1,vote,organism,gene_product,transcript_product
PF3D7_0100200.1-p1,MKIHYINILLFELPLNILIYNQRNHKSTTPHTPNHTQTTRLLCECE...,PF3D7_0100200.1-p1 | transcript=PF3D7_0100200....,331,110,Other,1.0,110,Other,1.000000,110,...,110,Other,1.000000,110,Other,1.000000,Other,Plasmodium_falciparum_3D7,rifin,rifin
PF3D7_0100400.1-p1,MKIHYTNILLFPLKLNILVNTHKKPSITSRHIQTTRLLCECELYTP...,PF3D7_0100400.1-p1 | transcript=PF3D7_0100400....,372,110,Other,1.0,110,Other,1.000000,110,...,110,Other,1.000000,110,Other,1.000000,Other,Plasmodium_falciparum_3D7,rifin,rifin
PF3D7_0100600.1-p1,MKLHYTKLLLFFFTLNILLTSYHAHNKNKPSITPHHTRSTTSRLLS...,PF3D7_0100600.1-p1 | transcript=PF3D7_0100600....,359,110,Other,1.0,110,Other,1.000000,110,...,110,Other,1.000000,110,Other,1.000000,Other,Plasmodium_falciparum_3D7,rifin,rifin
PF3D7_0100700.1-p1,MHDIGVALSSTDIEHTLNFYKLVKDGKSTDEMKNCIYAFIKYYDTL...,PF3D7_0100700.1-p1 | transcript=PF3D7_0100700....,67,110,Other,1.0,110,Other,1.000000,110,...,110,Other,1.000000,110,Other,1.000000,Other,Plasmodium_falciparum_3D7,"Plasmodium exported protein, unknown function,...","Plasmodium exported protein, unknown function,..."
PF3D7_0100800.1-p1,MKFHYINIFLFALPLNILVHNKRNHMKTIICTPKTKQTKTHRTLCE...,PF3D7_0100800.1-p1 | transcript=PF3D7_0100800....,344,110,Other,1.0,110,Other,1.000000,110,...,110,Other,1.000000,110,Other,1.000000,Other,Plasmodium_falciparum_3D7,rifin,rifin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PF3D7_API04500.1-p1,MIKINIIKKYYYKKYKNFFQKKIIINKKFILNKNLIIKLNFYEYIK...,PF3D7_API04500.1-p1 | transcript=PF3D7_API0450...,101,110,Other,1.0,110,Other,1.000000,110,...,110,Other,1.000000,110,Other,1.000000,Other,Plasmodium_falciparum_3D7,hypothetical protein,hypothetical protein
PF3D7_API04600.1-p1,MKYNKNVINTNFLYINKYKKLLKYKKNKNIIYILYKKYILILKKLI...,PF3D7_API04600.1-p1 | transcript=PF3D7_API0460...,51,110,Other,1.0,110,Other,1.000000,110,...,110,Other,1.000000,110,Other,1.000000,Other,Plasmodium_falciparum_3D7,hypothetical protein,hypothetical protein
PF3D7_API04700.1-p1,MIKLKNFLNIYNLNYKYQYKNKINLYLIRQGLNINLIKNLSSNIFL...,PF3D7_API04700.1-p1 | transcript=PF3D7_API0470...,470,110,Other,1.0,110,Other,0.999464,110,...,110,Other,0.999996,110,Other,0.999924,Other,Plasmodium_falciparum_3D7,iron-sulfur cluster assembly protein SufB,iron-sulfur cluster assembly protein SufB
PF3D7_MIT01400.1-p1,ILFSNLSNIKAHLVSYPALTSLYGTSLKYFSVGILFTFNPIILLIF...,PF3D7_MIT01400.1-p1 | transcript=PF3D7_MIT0140...,249,110,Other,1.0,110,Other,1.000000,110,...,110,Other,1.000000,110,Other,1.000000,Other,Plasmodium_falciparum_3D7,cytochrome c oxidase subunit 3,cytochrome c oxidase subunit 3


In [182]:
list_df_500

,seq,desc,len_seq,model_0_labels_1,model_0_family_1,model_0_prob_1,model_1_labels_1,model_1_family_1,model_1_prob_1,model_2_labels_1,...,model_3_labels_1,model_3_family_1,model_3_prob_1,model_4_labels_1,model_4_family_1,model_4_prob_1,vote,organism,gene_product,transcript_product
C4B63_1g1002-t42_1-p1,MAMMMTGRVLLVCALCVLWCGTSGRCDDAKQKELLDQRSVSPLGGE...,C4B63_1g1002-t42_1-p1 | transcript=C4B63_1g100...,380,110,Other,1.0,110,Other,1.000000,110,...,110,Other,1.000000,110,Other,1.000000,1.000000,Trypanosoma_cruzi_Dm28c_2018,Mucin-associated surface protein (MASP) [Sourc...,Mucin-associated surface protein (MASP)
C4B63_1g1003-t42_1-p1,MMTCRLLCALLVLALCCCPSVSAAGTQEAVKEPSKSTTTNDTTKAQ...,C4B63_1g1003-t42_1-p1 | transcript=C4B63_1g100...,224,110,Other,1.0,110,Other,1.000000,110,...,110,Other,1.000000,110,Other,1.000000,1.000000,Trypanosoma_cruzi_Dm28c_2018,Putative mucin TcMUCII [Source:UniProtKB/TrEMB...,mucin TcMUCII
C4B63_1g1004-t42_1-p1,MAMMMTGRVLLVCALCVLWCGAGGVYARYLDNNPVGGCMASGGFGA...,C4B63_1g1004-t42_1-p1 | transcript=C4B63_1g100...,420,110,Other,1.0,110,Other,1.000000,110,...,110,Other,1.000000,110,Other,1.000000,1.000000,Trypanosoma_cruzi_Dm28c_2018,Mucin-associated surface protein (MASP) [Sourc...,Mucin-associated surface protein (MASP)
C4B63_1g1025-t42_1-p1,MAMMMTGRVLLVCALCVLWCGAGEGGAEMNDETLVGSQHLTGVGEK...,C4B63_1g1025-t42_1-p1 | transcript=C4B63_1g102...,479,110,Other,1.0,110,Other,1.000000,110,...,110,Other,1.000000,110,Other,1.000000,1.000000,Trypanosoma_cruzi_Dm28c_2018,Mucin-associated surface protein (MASP) [Sourc...,Mucin-associated surface protein (MASP)
C4B63_1g1037-t42_1-p1,MAMMTGRVLLVCALCVLWCGAGCGFAKEKAVQDVQTDVSGDDERHK...,C4B63_1g1037-t42_1-p1 | transcript=C4B63_1g103...,412,110,Other,1.0,110,Other,1.000000,110,...,110,Other,1.000000,110,Other,1.000000,1.000000,Trypanosoma_cruzi_Dm28c_2018,Mucin-associated surface protein (MASP) [Sourc...,Mucin-associated surface protein (MASP)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PF3D7_API04500.1-p1,MIKINIIKKYYYKKYKNFFQKKIIINKKFILNKNLIIKLNFYEYIK...,PF3D7_API04500.1-p1 | transcript=PF3D7_API0450...,101,110,Other,1.0,110,Other,1.000000,110,...,110,Other,1.000000,110,Other,1.000000,1.000000,Plasmodium_falciparum_3D7,hypothetical protein,hypothetical protein
PF3D7_API04600.1-p1,MKYNKNVINTNFLYINKYKKLLKYKKNKNIIYILYKKYILILKKLI...,PF3D7_API04600.1-p1 | transcript=PF3D7_API0460...,51,110,Other,1.0,110,Other,1.000000,110,...,110,Other,1.000000,110,Other,1.000000,1.000000,Plasmodium_falciparum_3D7,hypothetical protein,hypothetical protein
PF3D7_API04700.1-p1,MIKLKNFLNIYNLNYKYQYKNKINLYLIRQGLNINLIKNLSSNIFL...,PF3D7_API04700.1-p1 | transcript=PF3D7_API0470...,470,110,Other,1.0,110,Other,0.999464,110,...,110,Other,0.999996,110,Other,0.999924,0.998924,Plasmodium_falciparum_3D7,iron-sulfur cluster assembly protein SufB,iron-sulfur cluster assembly protein SufB
PF3D7_MIT01400.1-p1,ILFSNLSNIKAHLVSYPALTSLYGTSLKYFSVGILFTFNPIILLIF...,PF3D7_MIT01400.1-p1 | transcript=PF3D7_MIT0140...,249,110,Other,1.0,110,Other,1.000000,110,...,110,Other,1.000000,110,Other,1.000000,1.000000,Plasmodium_falciparum_3D7,cytochrome c oxidase subunit 3,cytochrome c oxidase subunit 3


In [195]:
list_df_500[(list_df_500['organism']=='Trypanosoma_cruzi_Dm28c_2018')&
            (list_df_500['vote'].str.startswith('GT')) & (~list_df_500['transcript_product'].str.contains('glycosyltransferase')) ][
   ['vote','vote_mean','organism','gene_product','transcript_product']].sort_values(by='vote_mean').tail(50)

,vote,vote_mean,organism,gene_product,transcript_product
C4B63_139g61-t42_1-p1,GT4,0.688976,Trypanosoma_cruzi_Dm28c_2018,conserved hypothetical protein,conserved hypothetical protein
C4B63_251g1-t42_1-p1,GT91,0.733743,Trypanosoma_cruzi_Dm28c_2018,conserved hypothetical protein,conserved hypothetical protein
C4B63_76g39-t42_1-p1,GT67,0.740084,Trypanosoma_cruzi_Dm28c_2018,N-acetyltransferase domain-containing protein ...,N-acetyltransferase
C4B63_3g1018-t42_1-p1,GT1,0.741465,Trypanosoma_cruzi_Dm28c_2018,conserved hypothetical protein,conserved hypothetical protein
C4B63_223g9-t42_1-p1,GT2,0.743881,Trypanosoma_cruzi_Dm28c_2018,conserved hypothetical protein,conserved hypothetical protein
C4B63_158g50-t42_1-p1,GT2,0.751493,Trypanosoma_cruzi_Dm28c_2018,Putative glycine dehydrogenase [decarboxylatin...,glycine dehydrogenase [decarboxylating]
C4B63_21g191-t42_1-p1,GT2,0.751493,Trypanosoma_cruzi_Dm28c_2018,Putative glycine dehydrogenase [decarboxylatin...,glycine dehydrogenase [decarboxylating]
C4B63_35g178-t42_1-p1,GT71,0.782245,Trypanosoma_cruzi_Dm28c_2018,Putative expression site-associated gene (ESAG...,expression site-associated gene (ESAG-like) pr...
C4B63_153g21-t42_1-p1,GT90,0.797835,Trypanosoma_cruzi_Dm28c_2018,Putative expression site-associated gene (ESAG...,expression site-associated gene (ESAG-like) pr...
C4B63_27g241-t42_1-p1,GT2,0.822838,Trypanosoma_cruzi_Dm28c_2018,conserved hypothetical protein,conserved hypothetical protein


In [189]:
list_df_500[(list_df_500['organism']=='Plasmodium_falciparum_3D7')&(list_df_500['vote'].str.startswith('GT'))][
   ['vote','vote_mean','organism','gene_product','transcript_product']].sort_values(by='vote_mean')

,vote,vote_mean,organism,gene_product,transcript_product
PF3D7_1354600.1-p1,GT2,0.844943,Plasmodium_falciparum_3D7,"60S ribosomal protein L7-2, putative","60S ribosomal protein L7-2, putative"
PF3D7_0107400.1-p1,GT2,0.891298,Plasmodium_falciparum_3D7,"ATP synthase-associated protein, putative","ATP synthase-associated protein, putative"
PF3D7_0934200.1-p1,GT4,0.926330,Plasmodium_falciparum_3D7,"conserved protein, unknown function","conserved protein, unknown function"
PF3D7_0211600.1-p1,GT1,1.000000,Plasmodium_falciparum_3D7,UDP-N-acetylglucosamine transferase subunit AL...,UDP-N-acetylglucosamine transferase subunit AL...
PF3D7_0806400.1-p1,GT1,1.000000,Plasmodium_falciparum_3D7,UDP-N-acetylglucosamine transferase subunit AL...,UDP-N-acetylglucosamine transferase subunit AL...
PF3D7_0909200.1-p1,GT68,1.000000,Plasmodium_falciparum_3D7,GDP-fucose protein O-fucosyltransferase 2,GDP-fucose protein O-fucosyltransferase 2
PF3D7_1032400.1-p1,GT4,1.000000,Plasmodium_falciparum_3D7,phosphatidylinositol N-acetylglucosaminyltrans...,phosphatidylinositol N-acetylglucosaminyltrans...
PF3D7_1141600.1-p1,GT2,1.000000,Plasmodium_falciparum_3D7,dolichol-phosphate mannosyltransferase,dolichol-phosphate mannosyltransferase
PF3D7_1210900.1-p1,GT50,1.000000,Plasmodium_falciparum_3D7,GPI mannosyltransferase 1,GPI mannosyltransferase 1


In [188]:
list_df_500[(list_df_500['organism']=='Leishmania_donovani_BPK282A1')&(list_df_500['vote'].str.startswith('GT'))][
   ['vote','vote_mean','organism','gene_product','transcript_product']].sort_values(by='vote_mean')

,vote,vote_mean,organism,gene_product,transcript_product
LdBPK_280700.1.1-p1,GT2,0.640167,Leishmania_donovani_BPK282A1,"hypothetical protein, conserved","hypothetical protein, conserved"
LdBPK_090740.1.1-p1,GT2,0.712756,Leishmania_donovani_BPK282A1,"hypothetical protein, conserved","hypothetical protein, conserved"
LdBPK_212180.1.1-p1,GT8,0.770826,Leishmania_donovani_BPK282A1,"GPI transamidase component GAA1, putative","GPI transamidase component GAA1, putative"
LdBPK_061340.1.1-p1,GT1,0.830498,Leishmania_donovani_BPK282A1,protoporphyrinogen oxidase-like protein,protoporphyrinogen oxidase-like protein
LdBPK_367290.1.1-p1,GT90,0.858752,Leishmania_donovani_BPK282A1,delta8 fatty acid desaturase-like protein,delta8 fatty acid desaturase-like protein
LdBPK_020170.1.1-p1,GT67,0.933007,Leishmania_donovani_BPK282A1,"phosphoglycan beta 1,3 galactosyltransferase","phosphoglycan beta 1,3 galactosyltransferase"
LdBPK_363230.1.1-p1,GT25,0.956589,Leishmania_donovani_BPK282A1,"Octanoyltransferase, putative","Octanoyltransferase, putative"
LdBPK_030560.1.1-p1,GT2,0.994477,Leishmania_donovani_BPK282A1,"hypothetical protein, conserved","hypothetical protein, conserved"
LdBPK_281390.1.1-p1,GT49,0.997822,Leishmania_donovani_BPK282A1,"phenylalanine-4-hydroxylase, putative","phenylalanine-4-hydroxylase, putative"
LdBPK_020160.1.1-p1,GT67,0.998527,Leishmania_donovani_BPK282A1,"phosphoglycan beta 1,3 galactosyltransferase, ...","phosphoglycan beta 1,3 galactosyltransferase, ..."


In [177]:
list_df_500[(list_df_500['organism']=='Trypanosoma_brucei_brucei_TREU927')&(list_df_500['vote'].str.startswith('GT'))]

,seq,desc,len_seq,model_0_labels_1,model_0_family_1,model_0_prob_1,model_1_labels_1,model_1_family_1,model_1_prob_1,model_2_labels_1,...,model_3_labels_1,model_3_family_1,model_3_prob_1,model_4_labels_1,model_4_family_1,model_4_prob_1,vote,organism,gene_product,transcript_product
Tb927.2.1780:mRNA-p1,MRRHRVAVVSDFFYPGFGGVEVHIYSLGQCLMRRGHKVIVITRAYG...,Tb927.2.1780:mRNA-p1 | transcript=Tb927.2.1780...,455,47,GT4,1.000000,47,GT4,1.000000,47,...,47,GT4,1.000000,47,GT4,1.000000,GT4,Trypanosoma_brucei_brucei_TREU927,GlcNAc-PI synthesis protein,GlcNAc-PI synthesis protein
Tb927.2.2020:mRNA-p1,MRKRKNGKDAHFECRVGIAIVVWNMVLLSVCLVAVLLKNNQHDTKG...,Tb927.2.2020:mRNA-p1 | transcript=Tb927.2.2020...,383,110,Other,0.576993,54,GT47,0.997870,54,...,54,GT47,0.996256,101,GT90,0.929472,GT47,Trypanosoma_brucei_brucei_TREU927,expression site-associated gene 3 (ESAG3) prot...,expression site-associated gene 3 (ESAG3) prot...
Tb927.2.3370:mRNA-p1,MVSKGLQMVGSTRRKRIVLVVFLVFAVLFIAWINKQPKRKELTFQQ...,Tb927.2.3370:mRNA-p1 | transcript=Tb927.2.3370...,377,76,GT67,0.999966,76,GT67,1.000000,76,...,76,GT67,1.000000,76,GT67,1.000000,GT67,Trypanosoma_brucei_brucei_TREU927,UDP-Gal or UDP-GlcNAc-dependent glycosyltransf...,UDP-Gal or UDP-GlcNAc-dependent glycosyltransf...
Tb927.3.1870.2:mRNA-p1,MVSLNRKQKFQLAVPATLLLFLFVYVLLGGGSTQKSKRHGMHMDFN...,Tb927.3.1870.2:mRNA-p1 | transcript=Tb927.3.18...,289,39,GT31,0.999843,39,GT31,0.999810,39,...,110,Other,0.796591,39,GT31,0.999660,GT31,Trypanosoma_brucei_brucei_TREU927,"CHAP domain containing protein, putative","CHAP domain containing protein, putative"
Tb927.3.2390:mRNA-p1,MKILRDVYRQTFPKKLRFDKSSRILYAQIYVIARRHKHRKYIPKED...,Tb927.3.2390:mRNA-p1 | transcript=Tb927.3.2390...,442,110,Other,0.998957,77,GT68,0.552543,77,...,77,GT68,0.745396,110,Other,0.985904,GT68,Trypanosoma_brucei_brucei_TREU927,"hypothetical protein, conserved","hypothetical protein, conserved"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Tb09.v4.0080:mRNA-p1,MRFVYNAEERIGESEGHFVVAGIPSIDNDERFRRRKLQRETCWSYS...,Tb09.v4.0080:mRNA-p1 | transcript=Tb09.v4.0080...,327,76,GT67,1.000000,76,GT67,0.999997,76,...,76,GT67,1.000000,76,GT67,1.000000,GT67,Trypanosoma_brucei_brucei_TREU927,"hypothetical protein, conserved","hypothetical protein, conserved"
Tb10.v4.0188:mRNA-p1,MKNIERIYAIIGCLSFIVMVINILVSLYSLFGEDESISLAIAEDEY...,Tb10.v4.0188:mRNA-p1 | transcript=Tb10.v4.0188...,254,76,GT67,1.000000,76,GT67,0.999938,76,...,76,GT67,1.000000,76,GT67,1.000000,GT67,Trypanosoma_brucei_brucei_TREU927,hypothetical protein,hypothetical protein
Tb11.0730:mRNA-p1,MKCIRCIYAAVGCLSFIVMEINLLVSLYSLFSEGDTISLDIVEDEY...,Tb11.0730:mRNA-p1 | transcript=Tb11.0730:mRNA ...,171,76,GT67,1.000000,76,GT67,1.000000,76,...,76,GT67,0.999989,76,GT67,0.999999,GT67,Trypanosoma_brucei_brucei_TREU927,hypothetical protein,hypothetical protein
Tb11.0740:mRNA-p1,MKADDDIFMRVPLYLKYLELLPREKLNMGRAVGYPKEFSDEITWHT...,Tb11.0740:mRNA-p1 | transcript=Tb11.0740:mRNA ...,183,76,GT67,0.999539,76,GT67,0.999988,39,...,76,GT67,1.000000,76,GT67,1.000000,GT67,Trypanosoma_brucei_brucei_TREU927,hypothetical protein,hypothetical protein


In [174]:
ids='''Tb927.2.1780:mRNA-p1
Tb927.2.3370:mRNA-p1
Tb927.3.5660:mRNA-p1
Tb927.4.4250:mRNA-p1
Tb927.4.4270:mRNA-p1
Tb927.4.4290:mRNA-p1
Tb927.4.5240:mRNA-p1
Tb927.4.5250:mRNA-p1
Tb927.4.5260:mRNA-p1
Tb927.4.5270:mRNA-p1
Tb927.4.5280:mRNA-p1
Tb927.4.5290:mRNA-p1
Tb927.4.5300:mRNA-p1
Tb927.5.2760:mRNA-p1
Tb927.6.1960:mRNA-p1
Tb927.6.3300:mRNA-p1
Tb927.7.300:mRNA-p1
Tb927.8.7150:mRNA-p1
Tb927.8.8090:mRNA-p1
Tb927.8.8100:mRNA-p1
Tb927.9.3600:mRNA-p1
Tb927.9.800:mRNA-p1
Tb927.9.9200:mRNA-p1
Tb927.10.4610:mRNA-p1
Tb927.10.6530:mRNA-p1
Tb927.11.3900:mRNA-p1
Tb927.11.9990:mRNA-p1
Tb11.v5.0328.1-p1
Tb11.v5.0558.1-p1
Tb11.v5.0765.1-p1
Tb11.v5.0880.1-p1
Tb09.v4.0080:mRNA-p1
Tb11.0740:mRNA-p1
Tb927.10.12290:mRNA-p1
Tb10.v4.0188:mRNA-p1
Tb927.11.20410:mRNA-p1
Tb11.0730:mRNA-p1
Tb927.11.9360:mRNA-p1
Tb11.v5.0437.1-p1
Tb927.3.1870.2:mRNA-p1
Tb927.3.5410:mRNA-p1
Tb927.10.5740:mRNA-p1
Tb927.8.7140:mRNA-p1
Tb927.11.18780.1-p1
Tb11.1160:mRNA-p1
Tb927.2.2020:mRNA-p1
Tb927.11.14320:mRNA-p1
Tb927.9.16190:mRNA-p1
Tb927.10.11820:mRNA-p1
Tb927.10.3290:mRNA-p1
Tb927.5.3350:mRNA-p1
Tb927.1.2320:mRNA-p1
Tb927.10.120:mRNA-p1
Tb927.10.8010:mRNA-p1
Tb927.9.5800:mRNA-p1
Tb11.02.5105:mRNA-p1
Tb11.02.5105b.1-p1
Tb927.10.5080:mRNA-p1
Tb11.v5.0339.1-p1
Tb10.v4.0032:mRNA-p1
Tb11.v5.0922.1-p1
Tb927.10.4370:mRNA-p1
Tb927.4.5790:mRNA-p1
Tb927.11.7330:mRNA-p1
Tb927.10.9500:mRNA-p1
Tb11.v5.0814.1-p1
Tb927.3.820:mRNA-p1
Tb927.9.5000:mRNA-p1
Tb09.v4.0018:mRNA-p1
Tb927.9.15940:mRNA-p1
Tb927.4.3820:mRNA-p1
Tb11.v5.0278.1-p1
Tb927.10.4520:mRNA-p1'''.split()
list_df_500.loc[ids].tail(50)

,seq,desc,len_seq,model_0_labels_1,model_0_family_1,model_0_prob_1,model_1_labels_1,model_1_family_1,model_1_prob_1,model_2_labels_1,...,model_3_labels_1,model_3_family_1,model_3_prob_1,model_4_labels_1,model_4_family_1,model_4_prob_1,vote,organism,gene_product,transcript_product
Tb927.10.4610:mRNA-p1,MTVKYSIIVPAYKECGNLEPLTKQVFDALADDGFSKNEVEMVIVDD...,Tb927.10.4610:mRNA-p1 | transcript=Tb927.10.46...,259,26,GT2,1.000000,26,GT2,1.000000,26,...,26,GT2,1.000000,26,GT2,1.000000,GT2,Trypanosoma_brucei_brucei_TREU927,"dolicholphosphate-mannose synthase, putative","dolicholphosphate-mannose synthase, putative"
Tb927.10.6530:mRNA-p1,MEYRWLNRVLTVELCMSIIAFHLVHFTGIDWNAYMQEVKGFLDGEL...,Tb927.10.6530:mRNA-p1 | transcript=Tb927.10.65...,404,66,GT58,1.000000,66,GT58,1.000000,66,...,66,GT58,1.000000,66,GT58,1.000000,GT58,Trypanosoma_brucei_brucei_TREU927,Dolichyl-P-Man:Man(5)GlcNAc(2)-PP-dolichyl alp...,Dolichyl-P-Man:Man(5)GlcNAc(2)-PP-dolichyl alp...
Tb927.11.3900:mRNA-p1,MFCRSVKWFSVFDACYVLNLDRRQDRWAHVQQQLSRAKLETFLRPP...,Tb927.11.3900:mRNA-p1 | transcript=Tb927.11.39...,275,32,GT25,1.000000,32,GT25,1.000000,32,...,32,GT25,1.000000,32,GT25,1.000000,GT25,Trypanosoma_brucei_brucei_TREU927,"glycosyltransferase family-like protein, putative","glycosyltransferase family-like protein, putative"
Tb927.11.9990:mRNA-p1,MSLPRRMCMFATCQSNRKSCSMRQSSVTSEGYRYSSTFAAGRLFDA...,Tb927.11.9990:mRNA-p1 | transcript=Tb927.11.99...,307,32,GT25,1.000000,32,GT25,1.000000,32,...,32,GT25,1.000000,32,GT25,1.000000,GT25,Trypanosoma_brucei_brucei_TREU927,glycosyl transferase-like protein,glycosyl transferase-like protein
Tb11.v5.0328.1-p1,RDFRRRNIQRETCWSYPEVARKSNNFTGKLLVVYALSPYTNNNCKL...,Tb11.v5.0328.1-p1 | transcript=Tb11.v5.0328.1 ...,292,76,GT67,1.000000,76,GT67,1.000000,76,...,76,GT67,1.000000,76,GT67,1.000000,GT67,Trypanosoma_brucei_brucei_TREU927,UDP-Gal or UDP-GlcNAc-dependent glycosyltransf...,UDP-Gal or UDP-GlcNAc-dependent glycosyltransf...
Tb11.v5.0558.1-p1,MTKGGKVAVTKGSAQSDGAGEGGMSKAKSSTTFVATGGGSLPAWAL...,Tb11.v5.0558.1-p1 | transcript=Tb11.v5.0558.1 ...,460,75,GT66,1.000000,75,GT66,1.000000,75,...,75,GT66,1.000000,75,GT66,1.000000,GT66,Trypanosoma_brucei_brucei_TREU927,"oligosaccharyl transferase subunit, putative","oligosaccharyl transferase subunit, putative"
Tb11.v5.0765.1-p1,MRRHRVAVVSDFFYPGFGGVEVHIYSLGQCLMRRGHKVIVITRAYG...,Tb11.v5.0765.1-p1 | transcript=Tb11.v5.0765.1 ...,428,47,GT4,1.000000,47,GT4,1.000000,47,...,47,GT4,1.000000,47,GT4,1.000000,GT4,Trypanosoma_brucei_brucei_TREU927,N-acetylglucosaminyl-phosphatidylinositol bios...,N-acetylglucosaminyl-phosphatidylinositol bios...
Tb11.v5.0880.1-p1,MSLLIICCNLHRIHEVSTFLDENQAGVSQPIDEGEYLMFVPSNVAA...,Tb11.v5.0880.1-p1 | transcript=Tb11.v5.0880.1 ...,373,76,GT67,1.000000,76,GT67,1.000000,76,...,76,GT67,1.000000,76,GT67,1.000000,GT67,Trypanosoma_brucei_brucei_TREU927,UDP-Gal or UDP-GlcNAc-dependent glycosyltransf...,UDP-Gal or UDP-GlcNAc-dependent glycosyltransf...
Tb09.v4.0080:mRNA-p1,MRFVYNAEERIGESEGHFVVAGIPSIDNDERFRRRKLQRETCWSYS...,Tb09.v4.0080:mRNA-p1 | transcript=Tb09.v4.0080...,327,76,GT67,1.000000,76,GT67,0.999997,76,...,76,GT67,1.000000,76,GT67,1.000000,GT67,Trypanosoma_brucei_brucei_TREU927,"hypothetical protein, conserved","hypothetical protein, conserved"
Tb11.0740:mRNA-p1,MKADDDIFMRVPLYLKYLELLPREKLNMGRAVGYPKEFSDEITWHT...,Tb11.0740:mRNA-p1 | transcript=Tb11.0740:mRNA ...,183,76,GT67,0.999539,76,GT67,0.999988,39,...,76,GT67,1.000000,76,GT67,1.000000,GT67,Trypanosoma_brucei_brucei_TREU927,hypothetical protein,hypothetical protein


In [134]:
list_df_500.loc['Tb927.9.16190:mRNA-p1']

seq                   MISRKTYLWFKFEVETLKPTNPYKMKADDDIFMRAPLYLKYLEVLP...
desc                  Tb927.9.16190:mRNA-p1 | transcript=Tb927.9.161...
len_seq                                                             203
model_0_labels_1                                                     76
model_0_family_1                                                   GT67
model_0_prob_1                                                 0.987876
model_1_labels_1                                                     76
model_1_family_1                                                   GT67
model_1_prob_1                                                  0.99881
model_2_labels_1                                                    110
model_2_family_1                                                  Other
model_2_prob_1                                                 0.976914
model_3_labels_1                                                     76
model_3_family_1                                                

In [176]:
#list_df_500[['model_0_family_1','model_1_family_1','model_2_family_1','model_3_family_1','model_4_family_1']]

In [150]:
def select(X):
    temp = []
    items = list(set(X))
    to_count = list(X)
    for n in items:
        temp.append((to_count.count(n),n))
    temp.sort()
    return temp[-1][-1]


X=['Other','mio','Other','Other','Other']   
select(X)        

'Other'